In [7]:
import pandas as pd
import os

from google.colab import drive
drive.mount('/content/drive')

class Activity():
    """
    Clase que representa una actividad en el proyecto.
    """
    def __init__(self, name, duration):
        """
        Inicializa una actividad con un nombre y una duración.

        Args:
            name (str): Nombre de la actividad.
            duration (int): Duración de la actividad en semanas.
        """
        self.name = name
        self.duration = duration

class DirectedGraph():
    """
    Clase que representa una gráfica dirigida.
    """
    def __init__(self):
        """
        Inicializa una gráfica dirigida con un diccionario vacío para almacenar los nodos y los arcos.
        """
        self.nodes = {}

    def add_vertex(self, vertex):
        """
        Agrega un nodo a la gráfica si no existe.

        Args:
            vertex (str): Nombre del nodo a agregar.
        """
        if vertex not in self.nodes:
            self.nodes[vertex] = []

    def add_edge(self, start_vertex, end_vertex, duration):
        """
        Agrega un arco a la gráfica, creando los nodos si es necesario.

        Args:
            start_vertex (str): Nodo de inicio del arco.
            end_vertex (str): Nodo de fin del arco.
            duration (int): Duración del arco en semanas.
        """
        if start_vertex not in self.nodes:
            self.add_vertex(start_vertex)
        if end_vertex not in self.nodes:
            self.add_vertex(end_vertex)
        self.nodes[start_vertex].append((end_vertex, duration))

    def predecessors(self, vertex):
        """
        Devuelve una lista de los predecesores del nodo dado.

        Args:
            vertex (str): Nodo para el que se buscan predecesores.

        Returns:
            list: Lista de predecesores del nodo dado.
        """
        return [key for key in self.nodes if vertex in [edge[0] for edge in self.nodes[key]]]

    def successors(self, vertex):
        """
        Devuelve una lista de los sucesores del nodo dado.

        Args:
            vertex (str): Nodo para el que se buscan sucesores.

        Returns:
            list: Lista de sucesores del nodo dado.
        """
        return [edge[0] for edge in self.nodes[vertex]]

class Proyect():
    """
    Clase que gestiona la programación del proyecto.
    """
    def __init__(self, file_path):
        """
        Inicializa el programador de proyectos con la ruta del archivo de entrada.

        Args:
            file_path (str): Ruta del archivo de Excel que contiene los datos del proyecto.
        """
        self.file_path = file_path
        self.data = None
        self.graph = DirectedGraph()

    def read_excel_data(self):
        """
        Lee los datos del archivo de Excel y carga en el atributo 'data'.
        """
        if os.path.exists(self.file_path):
            self.data = pd.read_excel(self.file_path)
        else:
            print(f"El archivo '{self.file_path}' no existe.")

    def build_graph(self):
        """
        Construye una gráfica dirigida a partir de los datos cargados del archivo de Excel.
        """
        if self.data is not None:
            for row in self.data.itertuples():
                if pd.notna(row.Precedentes):
                    precedents = str(row.Precedentes).split(',')
                    for precedent in precedents:
                        if precedent.strip() != "":
                            duration = row.Duracion
                            self.graph.add_edge(precedent.strip(), str(row.Descripcion), duration)

    def earliest_times(self):
        """
        Calcula las fechas más tempranas para cada nodo en la gráfica.
        """
        earliest_times = {node: 0 for node in self.graph.nodes.keys()}
        for node in self.graph.nodes.keys():
            for predecesor in self.graph.predecessors(node):
                for edge in self.graph.nodes[predecesor]:
                    if edge[0] == node:
                        earliest_times[node] = max(earliest_times[node], earliest_times[predecesor] + edge[1])
        return earliest_times

    def latest_times(self, earliest_times):
        """
        Calcula las fechas más tardías para cada nodo en la gráfica.

        Args:
            earliest_times (dict): Diccionario que contiene las fechas más tempranas para cada nodo.

        Returns:
            dict: Diccionario que contiene las fechas más tardías para cada nodo.
        """
        latest_times = {node: earliest_times[node] for node in self.graph.nodes.keys()}
        for node in reversed(list(self.graph.nodes.keys())):
            for successor in self.graph.successors(node):
                for edge in self.graph.nodes[node]:
                    if edge[0] == successor:
                        latest_times[node] = min(latest_times[node], latest_times[successor] - edge[1])
        return latest_times

    def critical_paths(self):
        """
        Calcula el camino crítico del proyecto.

        Returns:
            tuple: Una tupla que contiene el tiempo mínimo necesario para la elaboración del proyecto
                y las actividades críticas.
        """
        earliest_start_time = self.earliest_times()
        latest_start_time = self.latest_times(earliest_start_time)
        critical_activities = []
        for node in self.graph.nodes.keys():
            if earliest_start_time[node] == latest_start_time[node]:
                critical_activities.append((node, earliest_start_time[node]))
        min_time = max(latest_start_time.values())
        return min_time, critical_activities

    def report(self, min_time, critical_activities, output_file):
        """
        Genera un informe de salida que muestra el tiempo mínimo necesario para la elaboración del proyecto
        y las actividades críticas.

        Args:
            min_time (int): Tiempo mínimo necesario para la elaboración del proyecto.
            critical_activities (list): Lista de tuplas que contienen el nombre y la duración de las actividades críticas.
            output_file (str): Ruta del archivo de salida donde se escribirá el informe.
        """
        with open(output_file, "w") as file:
            file.write(f"El tiempo necesario para la elaboracion del proyecto es de {min_time} semanas.\n")
            file.write("Las actividades criticas son:\n")
            for idx, (activity, duration) in enumerate(critical_activities, 1):
                file.write(f"{idx}. {activity}: {duration} semanas\n")

    def run(self, output_file):
        """
        Ejecuta todo el proceso de programación del proyecto.

        Args:
            output_file (str): Ruta del archivo de salida donde se escribirá el informe.
        """
        self.read_excel_data()
        self.build_graph()
        min_time, critical_activities = self.critical_paths()
        self.report(min_time, critical_activities, output_file)

def main():
    """
    Función principal que coordina la ejecución del programa.
    """
    excel_file = '/content/drive/My Drive/Programacion24-2/datos_proyecto.xlsx'
    output_file = '/content/drive/My Drive/Programacion24-2/reporte.txt'

    proyecto = Proyect(excel_file)
    proyecto.run(output_file)

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
